In [1]:
import stim
from stimrgs_v1.utils import *
from stimrgs_v1.stabilizer_operations import *
from tqdm import tqdm

In [3]:
# Configuration parameters
num_trial = 5
num_nodes = 32
num_bell_between_row = [i+2 for i in range(num_nodes//4 -1)]

stabilizers_results = []
stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

for num_bell in num_bell_between_row:

    # Reset max_found_bells every time we change num_bell_between_row
    max_found_bells = 0

    for _ in range(num_trial):

        stabilizers_results = []
        stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

        for num_measurements in range(5, num_nodes - 3):
            # print(f'Measure: {num_measurements} nodes')
            combinations = generate_combinations_with_adjustable_replacement(
                nodes=[i for i in range(num_nodes)], 
                max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
                max_self_combination=1
            )    

            for c in combinations:

                # Restart parameters before operate new measurement set
                found: bool = False
                bell_found_list = []

                q, circuit = generate_rgs_random(num_nodes=num_nodes, num_bell_between_row=num_bell)
                s = stim.TableauSimulator()
                s.do_circuit(stim.Circuit(circuit))

                # Operate measurement
                for index in c:
                    # Update s, q
                    s, q = measure_z_with_correction(index=index, s=s, queries=q)

                # Check if result match one of all possible results
                for result in stabilizers_results:
                    if is_subset(result, q):
                        # print(f"Found solution: {result}")   
                        bell_found_list.append(result)                     
                        found = True                       
                
                # Update when found new high
                if len(bell_found_list) >= max_found_bells:
                    max_found_bells = len(bell_found_list)

    print(f'Max found bells {num_bell}: {max_found_bells}')

KeyboardInterrupt: 